## Notebook to download fractional snow-covered area (fSCA) products from MODIS over a pre-defined Area of Interest (AOI) and time period through the Google Earth Engine (GEE) data repository

Rainey Aberle

Last edited: July 2022

### Requirements:
- __GEE account__: Sign up for your free account [HERE](https://earthengine.google.com/new_signup/). Once you have an account, you will be asked to authorize your GEE account for use within this notebook in Step 3 below. 
- __AOI shapefile__: define the path in your local directory to the AOI `.shp` file in Step 2 below. 

### 1. Import packages

In [ ]:
import ee
import geemap
import geopandas as gpd
import os
import rasterio as rio

### 2. Set file locations and search settings (MODIFY THIS SECTION)

_MODIS specifications (see more info [here](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/modis-overview/#:~:text=It%20has%20a%20viewing%20swath,500%20m%2C%20and%201%2C000%20m.))_
- Temporal coverage: Terra = 1999-12-18 to present, Aqua = 2002-05-04 to present
- Spatial coverage: Global

In [ ]:
# -----Set path to output folder for image downloads and figures
images_out_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/study-sites/Wolverine/imagery/MODIS/'
figures_out_path = images_out_path+'../../figures/'

# -----Set path to Area of Interest (AOI) shapefile
# include full path to file
AOI_fn = '/Users/raineyaberle/Research/PhD/GIS_data/RGI_outlines/Wolverine_RGI.shp' 

# -----Set temporal filters
# Start and end dates
# Format: 'YYYY-MM-DD'
start_date = '2016-01-01'
end_date = '2016-12-01'
# Calendar months to include in search (e.g., January = 1, December = 12)
# To include all months, set as: start_month = 1, end_month = 12
start_month = 4
end_month = 10

# -----Add spatial buffer to AOI
# buffer the area around the AOI for cropping. For no buffer, set to 0. 
buffer = 1000 # [m]

### 2. Authenticate and initialize GEE

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back in this notebook when prompted. 

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

### 3. Conduct search for MODIS fSCA data products

In [ ]:
# -----Load AOI from file and reformat for clipping
# read as gpd.DataFrame
AOI = gpd.read_file(AOI_fn) 
# reproject AOI to WGS 84 for compatibility
AOI_WGS = AOI.to_crs(4326)
# reformat AOI_UTM bounding box as ee.Geometry for clipping
AOI_WGS_bb_ee = ee.Geometry({"type": "Polygon","coordinates":
                             [[[AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]],
                               [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.miny[0]],
                               [AOI_WGS.geometry.bounds.maxx[0], AOI_WGS.geometry.bounds.maxy[0]],
                               [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.maxy[0]],
                               [AOI_WGS.geometry.bounds.minx[0], AOI_WGS.geometry.bounds.miny[0]]]
                             ]}).buffer(buffer) 

# -----Query GEE for Terra and Aqua fSCA image collections
Terra_fSCA = (ee.ImageCollection("MODIS/006/MOD10A1")
              .filterBounds(AOI_WGS_bb_ee) # spatial filter
              .filterDate(start_date, end_date) # date filter
              .filter(ee.Filter.calendarRange(start_month, end_month, 'month'))) # calendar month filter
Aqua_fSCA = (ee.ImageCollection("MODIS/006/MYD10A1")
              .filterBounds(AOI_WGS_bb_ee) # spatial filter
              .filterDate(start_date, end_date) # date filter
              .filter(ee.Filter.calendarRange(start_month, end_month, 'month'))) # calendar month filter
# print number of images in each collection
print(str(Terra_fSCA.size().getInfo()) + ' Terra fSCA images found.')
print(str(Aqua_fSCA.size().getInfo())  + ' Aqua fSCA images found.')

# -----Clip image collections to AOI
def clip_image_to_AOI(im, AOI=AOI_WGS_bb_ee):
    return im.clip(AOI)
Terra_fSCA_clipped = Terra_fSCA.map(clip_image_to_AOI)
Aqua_fSCA_clipped = Aqua_fSCA.map(clip_image_to_AOI)

# -----Display first image in each collection 
# define visualization parameters for snow cover images
snow_cover_vis = {'min': 0.0, 'max': 100.0, 'palette': ['white', 'blue']} 
# create GEE map and display
Map = geemap.Map(basemap='HYBRID')
Map.addLayerControl()
Map.addLayer(AOI_WGS_bb_ee, {'color': 'dd3497'}, 'AOI')
Map.addLayer(Terra_fSCA_clipped.first().select('NDSI_Snow_Cover'), snow_cover_vis, 'Terra fSCA')
Map.addLayer(Aqua_fSCA_clipped.first().select('NDSI_Snow_Cover'), snow_cover_vis, 'Aqua fSCA')
Map.centerObject(AOI_WGS_bb_ee, zoom=11)
Map

### 5. Download image collections to file

In [ ]:
# Export with WGS84 projection
crs='EPSG:4326' 
scale=500 # default scale [m]
# Terra
print('Downloading Terra fSCA images...')
Terra_fSCA_out_path = images_out_path + 'Terra_fSCA/'
geemap.ee_export_image_collection(Terra_fSCA_clipped, Terra_fSCA_out_path, crs=crs, scale=scale)
# Aqua
print('Aqua Terra fSCA images...')
Aqua_fSCA_out_path = images_out_path + 'Aqua_fSCA/'
geemap.ee_export_image_collection(Aqua_fSCA_clipped, Aqua_fSCA_out_path, crs=crs, scale=scale)

### _Optional:_ Create figures of the resulting image collections and compile into a .gif

In [ ]:
import os
import glob
import matplotlib.pyplot as plt

# -----Plot Aqua and Terra fSCA images captured the same day, save as .png files
# grab image file names
os.chdir(Terra_fSCA_out_path) # change directory
Terra_fSCA_fns = glob.glob('*.tif') # grab file names for all .tifs
os.chdir(Aqua_fSCA_out_path) # change directory
Aqua_fSCA_fns = glob.glob('*.tif') # grab file names for all .tifs
# determine unique image dates
fns = Terra_fSCA_fns + Aqua_fSCA_fns
fns_unique = list(set(fns))
fns_unique.sort() # sort chronologically
# reproject AOI to WGS84
AOI_rpj = AOI.to_crs(crs)
# loop through unique image dates
for fn in fns_unique:
    # set up figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))
    plt.rcParams.update({'font.size':14, 'font.sans-serif':'Arial'})
    # load and plot Terra fSCA image
    if os.path.exists(Terra_fSCA_out_path + fn):
        # open file
        im = rio.open(Terra_fSCA_out_path + fn)
        # load fSCA band
        fSCA = im.read(1)
        # define coordinates grid
        im_x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(fSCA)[1])
        im_y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(fSCA)[0])
        ax1.imshow(fSCA, cmap='Blues', clim=(0,100),
                   extent=(np.min(im_x), np.max(im_x), np.min(im_y), np.max(im_y)))
        AOI_rpj.plot(ax=ax1, facecolor='none', edgecolor='white', label='AOI')
        ax1.grid()
        # define axis ticks
        xticks = np.linspace(np.round(im_x[0], 3), np.round(im_x[-1], 3), num=5)
        yticks = np.linspace(np.round(im_y[0], 3), np.round(im_y[-1], 3), num=5)
        ax1.set_xticks(xticks)
        ax1.set_yticks(yticks)
        ax1.set_title('Terra')
    # load and plot Aqua fSCA image
    if os.path.exists(Aqua_fSCA_out_path + fn):
        # open file
        im = rio.open(Aqua_fSCA_out_path + fn)
        # load fSCA band
        fSCA = im.read(1)
        # define coordinates grid
        im_x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(fSCA)[1])
        im_y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(fSCA)[0])
        plot_Aqua = ax2.imshow(fSCA, cmap='Blues', clim=(0,100),
                   extent=(np.min(im_x), np.max(im_x), np.min(im_y), np.max(im_y)))
        AOI_rpj.plot(ax=ax2, facecolor='none', edgecolor='white', label='AOI')
        ax2.grid()
        # define axis ticks
        xticks = np.linspace(np.round(im_x[0], 3), np.round(im_x[-1], 3), num=5)
        yticks = np.linspace(np.round(im_y[0], 3), np.round(im_y[-1], 3), num=5)
        ax2.set_xticks(xticks)
        ax2.set_yticks(yticks)
        ax2.set_title('Aqua')
    fig.colorbar(plot_Aqua, ax=ax2, shrink=0.5, label='fSCA [%]')
    fig.suptitle(fn[0:4]+'-'+fn[5:7]+'-'+fn[8:10])
    fig.tight_layout()
    plt.show()
    fig_fn = fn[:-4]+'_MODIS_fSCA.png'
    # make directory for figures if it doesn't exist
    if os.path.exists(figures_out_path)==False:
        os.mkdir(figures_out_path)
        print('Made directory for output figures: ' + figures_out_path)
    fig.savefig(figures_out_path + fig_fn, dpi=150, facecolor='white')
    print('Figure saved to file: ' + figures_out_path + fig_fn)

In [ ]:
from PIL import Image as PIL_Image
from IPython.display import Image as IPy_Image

# -----Make a .gif of output images
os.chdir(figures_out_path)
fig_fns = glob.glob('*MODIS_fSCA.png') # load all output figure file names
fig_fns.sort() # sort chronologically
# grab figures date range for .gif file name
fig_start_date = fig_fns[0][:-4] # first figure date
fig_end_date = fig_fns[-1][:-4] # final figure date
frames = [PIL_Image.open(im) for im in fig_fns]
frame_one = frames[0]
gif_fn = (fig_start_date[0:4] + fig_start_date[5:7] + fig_start_date[8:10] + '_' + 
         fig_end_date[0:4] + fig_end_date[5:7] + fig_end_date[8:10] + 
          '_MODIS_fSCA.gif' )
frame_one.save(figures_out_path + gif_fn, format="GIF", append_images=frames, save_all=True, duration=1000, loop=0)
print('GIF saved to file:' + figures_out_path + gif_fn)

# -----Display .gif
IPy_Image(filename=figures_out_path+gif_fn)

In [ ]:
# -----Clean up: delete individual figure files
files = os.listdir(figures_out_path)
for item in files:
    if item.endswith("MODIS_fSCA.png"):
        os.remove(os.path.join(figures_out_path, item))
print('Individual figure files deleted.')